Testing Propotype with Selenium & google library
*** The Following must using selenium with FireFox web driver. 
*** Be sure that Firefox has been installed on the local machine.
*** The Script may only running on LOCAL MACHINE ONLY

In [ ]:
# Install Library
pip install selenium pandas google


In [2]:
from googlesearch import search
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from urllib.parse import urlparse
import pandas as pd
import time
import traceback
import re

OUTPUT_FILE = 'output.csv'
AUTHOR_MAX_WORDS = 3
SLEEP_INTERVAL = 2

p_elementList= []

# Cuntion for Getting Date from Content
def extractDate(string):
    try:
       # Define a regular expression for the date format DD/MM/YYYY and optional time
        date_time_pattern = re.compile(
            r"^\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b"  # Matches dates like DD/MM/YYYY at the start of the string
            r"|^\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s+\d{4}\b"  # Matches dates like December 20, 2023
            r"|^\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\d{4}\b"  # Matches dates like December 20, 2023
            r"|^\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2},\s+\d{4}\b"  # Matches dates like Dec 20, 2023
            r"|^\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s+\d{1,2},\d{4}\b"  # Matches dates like Dec 20, 2023
            r"(?:\s+\b\d{1,2}:\d{2}(?::\d{2})?\s*(?:AM|PM)?\s*(?:ET|WT)?\b)?$",  # Optionally matches times with optional AM/PM and time zone
            re.IGNORECASE  # Make the pattern case-insensitive
        )
        # Check if the entire string matches the pattern
        match = date_time_pattern.match(string)
        return bool(match)
    except:
        print("Error accessing a text:", e)
    return False

# Function for Getting Content from url
def getContent(driver,url):
    try:
        parsed_url = urlparse(url)

        authors = '' 
        source_domain = parsed_url.netloc
        copyright = ''
        paragraph = ''
        title = ''
        date = ''
        driver.get(url)
        H1_elements = driver.find_elements(By.TAG_NAME, "h1")
        p_elements = driver.find_elements(By.TAG_NAME, "p")
            
        # Getting Title
        for h1 in H1_elements:
            try:
                title = h1.text
                break  # Assuming you only need the first h1's text
            except Exception as e:
                print("Error accessing h1 text:", e)
                continue

        # Getting Content
        for p in p_elements:
            try:
                value = p.text
                # Filter out authors
                 # adding your owner key word here to filter out the authors
                if ((value.lower().strip().startswith('written by:')) or\
                    (value.lower().strip().startswith('article by:'))
                    ):
                    value.replace('Article by:', '')
                    authors = value.replace('Written by:', '') 
                    #authors = value.lower().replace('by ', '') 
                    #words = authors.split()
                    #if len(words) <= AUTHOR_MAX_WORDS and not authors:
                    #    authors = value
                    #else:
                    #    authors = ''
                    continue
                # Filter out Copyright
                if '©' in value or 'All Rights Reserved' in value:
                    copyright = value
                    continue

                # call date function to extract the date
                 # adding your owner key word here to filter out the published date
                if ((value.lower().strip().startswith('updated on')) or \
                    (value.lower().strip().startswith('published on')) or \
                    (value.lower().strip().startswith('published online')) 
                     ):
                    value = value.replace('Updated on', '').replace(':', '') 
                    value = value.replace('Published on', '').replace(':', '') 
                    value = value.replace('Published Online', '').replace(':', '') 
                    if (extractDate(value.strip())):
                        date = value
                        continue

                paragraph += value + ' '
            except Exception as e:
                print(f"Error accessing paragraph text from {url}:\n", e)
                continue
    except TimeoutException:
        authors = ''
        print(f"Timeout waiting for elements on the page: {url}")
    except NoSuchElementException:
        authors = ''
        print(f"An element was not found on the page: {url}")
    except Exception as e:
        authors = ''
        print(f"An unexpected error occurred with URL => {url}: {e}")

    return {'authors': authors, 'date' : date, 'title' : title, 'content': paragraph, 'source_domain': source_domain, 'url' : url, 'copyright': copyright } 

def save_to_csv_with_pandas(recordSet, filename):
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(recordSet)
    # Save the DataFrame to a CSV file
    df.to_csv(filename, index=False, encoding='utf-8')


C:\Users\kenne\AppData\Local\Temp\ipykernel_13012\2857286026.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Please run and input your query

In [3]:
print("Which query content do you want to search?")
queryString = input()

#Testing query
#queryString = 'Did Joe Biden tell voters to stay home and not vote?'
#queryString = 'Is Michael Saylor saying he will give away free bitcoin?'
#queryString = 'Is Taylor Swift going to give away a free Le Creuset cookware set?'
#queryString = 'Did Elon Musk say you can control a computer mouse with neuralink?'
#queryString = 'Oprah winfrey weight loss gummies?'
#queryString = 'Did Justin Trudeau say invest in Petro canada?'
#queryString = 'How to enter Jennifer Aniston Macbook giveaway?'

Which query content do you want to search?


Run following command to get result from google search

In [4]:
#Search in google url with google library
results = search(query=queryString,num = 10 )

Run following script to get DataFrame result
** you may keep tracing and use the mouse in firefox sometime for simlutate for human behavior

In [6]:
dataSet = []
driver = webdriver.Firefox()
i = 0
for url in results:
    print(f'Scripting from => {url}')
    try:
        data = getContent(driver,url)
        print(data)
        if (data['title'] !='' or data['content'] != ''):
            content = data['content']
            # adding your owner key word here to filter content
            if ((content.strip().startswith('Log In')) or \
                (content.lower() in 'live sex')
            ):
                print("content Excluded!!")
            else:
                dataSet.append(data)
    except:
        print(f"An unexpected error occurred with URL => {url}: {e}")
    #time.sleep(SLEEP_INTERVAL)
driver.close()
df = pd.DataFrame(dataSet)


Scripting from => https://malwaretips.com/blogs/jennifer-aniston-macbook-giveaway-scam/#overview_of_the_jennifer_aniston_macbook_giveaway_scam
{'authors': ' Thomas Orsolya', 'date': ' December 20, 2023', 'title': 'Don’t Get Scammed By The Fake Jennifer Aniston MacBook Giveaway', 'content': 'A fraudulent online campaign claiming beloved actress Jennifer Aniston is gifting free MacBook laptops has recently infiltrated social media through manipulated video ads and questionable website portals. However, the supposed giveaway actually intends stealing personal information and money rather than magnanimous charity. This article will break down exactly how the devious scam ensnares victims step-by-step alongside protection tips to avoid potential damages.   This Article Contains: A fraudulent online campaign claiming universally-beloved comedy icon Jennifer Aniston has teamed up with Apple facilitating an epic giveaway gifting free MacBook laptops aims exploiting fans through deception and p

In [7]:
df

,authors,date,title,content,source_domain,url,copyright
0,Thomas Orsolya,"December 20, 2023",Don’t Get Scammed By The Fake Jennifer Aniston...,A fraudulent online campaign claiming beloved ...,malwaretips.com,https://malwaretips.com/blogs/jennifer-aniston...,
1,Thomas Orsolya,"December 20, 2023",Don’t Get Scammed By The Fake Jennifer Aniston...,A fraudulent online campaign claiming beloved ...,malwaretips.com,https://malwaretips.com/blogs/jennifer-aniston...,
2,,"Feb 17,2024",Unveiling the Deceptive Jennifer Aniston MacBo...,"In today's digital age, s...",www.toolify.ai,https://www.toolify.ai/ai-news/unveiling-the-d...,Copyright ©2024 toolify
3,Thomas Orsolya,"December 20, 2023",Don’t Fall for the Fake Apple MacBook Giveaway...,A fraudulent online campaign falsely claiming ...,malwaretips.com,https://malwaretips.com/blogs/apple-macbook-gi...,
4,,,YouTube scam features fake Jennifer Aniston,"The latest in celebrity-impersonating scams, ...",cybernews.com,https://cybernews.com/news/youtube-scam-featur...,© 2024 Cybernews – Latest Cybersecurity and Te...
...,...,...,...,...,...,...,...
134,,,annablisss,My Specifics: I Do in My Shows: I Exclusi...,www.tiktok.com,https://www.tiktok.com/@.foodgotmewasted/video...,
135,,,Who wants a copy for their shelf??,Switch between light or dark mode here Log in ...,www.tiktok.com,https://www.tiktok.com/@willowwintersauthor/vi...,
136,,,Bonnie_petite's Bio and Free Webcam,Hey guys ^_^ I'm super new to this camming ...,www.tiktok.com,https://www.tiktok.com/@dailymealideas__/video...,© Copyright Chaturbate.com 2011 - 2024. All Ri...
137,,,"""Jennifer Aniston‪""‬\nSmartLess",44 min The one-and-only Jennifer Aniston joi...,podcasts.apple.com,https://podcasts.apple.com/us/podcast/jennifer...,


In [19]:
#export Dataframe to CSV if you need
save_to_csv_with_pandas(df, queryString.replace('?','.')+ 'csv')